In [10]:
from utils.models_dot import GMF, MLP, NeuFM
from utils.dataset import RatingsDataset
from torch.utils.data import DataLoader

from tqdm.notebook import tqdm

import math

import os

import pandas as pd

## Load

In [2]:
run = 1

In [3]:
users = 3974
movies = 3564

train_dataloader = DataLoader(
    RatingsDataset(
        "train_data/train.csv",
        "user_id",
        "movie_id",
        "rating",
    ), 
    batch_size=1024,
    num_workers=8,
    shuffle=True,
)
val_dataloader = DataLoader(
    RatingsDataset(
        "train_data/validation.csv",
        "user_id",
        "movie_id",
        "rating",
    ),
    batch_size=1024,
    num_workers=4,
    shuffle=True,
)

In [4]:
import torch
import torch.nn as nn
import torch.optim as opt

In [5]:
def define_model(k_gmf, k_mlp, layer_sizes, alpha=0.5):
    return NeuFM(
        GMF(users, movies, k_gmf),
        MLP(users, movies, k_mlp, layer_sizes),
        alpha=alpha,
    ).cuda()

In [6]:
def train(model, lr, epochs, train_dataloader, val_dataloader=None):
    criterion = nn.MSELoss().cuda()
    optimizer = opt.Adam(model.parameters(), lr=lr)
    
    model_state = None
    best_epoch = 0
    prev_val_loss = math.inf
    for epoch in tqdm(range(epochs)):
        n_batches = len(train_dataloader)
        avg_loss = 0
        val_loss = 0

        # Train step
        for i_batch, (vus, vis, rs) in enumerate(train_dataloader):
            vus = vus.cuda()
            vis = vis.cuda()
            rs = rs.cuda()

            optimizer.zero_grad()
            y_hat = model(vus, vis)

            loss = criterion(y_hat, rs)
            loss.backward()
            optimizer.step()

            avg_loss += math.sqrt(float(loss.detach().cpu()))
        
        avg_loss /= n_batches
        
        # Val step
        if val_dataloader is not None:
            with torch.no_grad():
                for val_vus, val_vis, val_rs in val_dataloader:
                    val_vus = val_vus.cuda()
                    val_vis = val_vis.cuda()
                    val_rs = val_rs.cuda()

                    val_pred = model(val_vus, val_vis)
                    val_loss += math.sqrt(float(criterion(val_pred, val_rs).detach().cpu()))

            val_loss /= len(val_dataloader)
            print(f"epoch: {epoch+1}, train_loss: {avg_loss}, val_loss: {val_loss}")
        else:
            val_loss = avg_loss
            print(f"epoch: {epoch+1}, train_loss: {avg_loss}")
                  
        if val_loss < prev_val_loss:
            prev_val_loss = val_loss
            model_state = model.state_dict()
            best_epoch = epoch
        
    return best_epoch, prev_val_loss, model_state

In [7]:
def fit(
    train_dataloader, val_dataloader,
    k_gmf, k_mlp, layer_sizes, alpha=0.5, lr=0.0005, epochs=40, 
    weight_path="/home/", run_number=1, random_state=None,
):
    if random_state is not None:
        torch.manual_seed(random_state)
        
    model = define_model(k_gmf, k_mlp, layer_sizes, alpha)
    
    best_epoch, val_loss, model_state = train(
        model, lr, epochs, train_dataloader, val_dataloader,
    )
    
    run_path = f"{weight_path}/run_{run_number}"
    if not os.path.isdir(run_path):
        os.makedirs(run_path)
    
    if val_dataloader is None:
        torch.save(
            model_state, 
            f"{run_path}/final-{best_epoch+1}-{val_loss}.pt"
        )
    else:
        torch.save(
            model_state, 
            f"{run_path}/{best_epoch+1}-{val_loss}.pt"
        )

In [ ]:
def predict(weight_path, k_gmf, k_mlp, layer_sizes, alpha, out_path, out_name):
    trained_model = NeuFM(
        GMF(users, movies, k_gmf),
        MLP(users, movies, k_mlp, layer_sizes),
        alpha,
    ).cuda()
    
    trained_model.load_state_dict(torch.load(weight_path))
    trained_model.cuda().eval()
    
    test_dataloader = DataLoader(
        RatingsDataset(
            "train_data/test.csv",
            "user_id",
            "movie_id",
        ), 
        batch_size=1024,
        num_workers=12
    )
    
    test_predictions = []

    for vus, vis in test_dataloader:
        vus = vus.cuda()
        vis = vis.cuda()

        pred = torch.clip(trained_model(vus, vis), 1, 5).cpu().ravel().tolist()
        test_predictions += pred
        
    test_csv = pd.read_csv("../../data/test_data.csv")
    
    out_df = pd.DataFrame.from_dict(
        {
            "id": list(test_csv["id"]),
            "rating": test_predictions
        }
    )

    out_df.to_csv(f"{out_path}/{out_name}.csv", index=False)
    out_df.head()

# Train

In [ ]:
fit(
    train_dataloader, val_dataloader,
    k_gmf=16,
    k_mlp=16,
    layer_sizes=[16, 8],
    alpha=0.5,
    lr=0.0005,
    epochs=34,
    weight_path="/home/nubol23/Documents/NCF_dot_weights",
    run_number=1,
    random_state=2
)

## Total fit

In [8]:
total_dataloader = DataLoader(
    RatingsDataset(
        "train_data/total.csv",
        "user_id",
        "movie_id",
        "rating",
    ), 
    batch_size=1024,
    num_workers=12,
    shuffle=True,
)

In [9]:
fit(
    total_dataloader, val_dataloader=None,
    k_gmf=16,
    k_mlp=16,
    layer_sizes=[16, 8],
    alpha=0.5,
    lr=0.0005,
    epochs=34,
    weight_path="/home/nubol23/Documents/NCF_dot_weights",
    run_number=1,
    random_state=2
)

  0%|          | 0/34 [00:00<?, ?it/s]

epoch: 1, train_loss: 2.5893365485190523
epoch: 2, train_loss: 0.9672088296570134
epoch: 3, train_loss: 0.9170506385237609
epoch: 4, train_loss: 0.9032267680396481
epoch: 5, train_loss: 0.8959118273966851
epoch: 6, train_loss: 0.8906218868014005
epoch: 7, train_loss: 0.8853005910353048
epoch: 8, train_loss: 0.8801576897917799
epoch: 9, train_loss: 0.8754042383557448
epoch: 10, train_loss: 0.870651483310532
epoch: 11, train_loss: 0.8660048420266693
epoch: 12, train_loss: 0.8617198281248419
epoch: 13, train_loss: 0.8574148942004435
epoch: 14, train_loss: 0.8533921645214392
epoch: 15, train_loss: 0.8495906874170289
epoch: 16, train_loss: 0.8456217221021054
epoch: 17, train_loss: 0.8416063523957125
epoch: 18, train_loss: 0.8375303340680198
epoch: 19, train_loss: 0.8336322232643073
epoch: 20, train_loss: 0.8292880757542056
epoch: 21, train_loss: 0.8249549034375587
epoch: 22, train_loss: 0.8208294865178158
epoch: 23, train_loss: 0.8165998957930868
epoch: 24, train_loss: 0.8123502329354026
ep

## Predict

In [12]:
predict(
    "/home/nubol23/Documents/NCF_dot_weights/run_1/final-34-0.774764311438812.pt",
    k_gmf=16,
    k_mlp=16,
    layer_sizes=[16, 8],
    alpha=0.5,
    out_path="/home/nubol23/Desktop/Codes/USP/SCC5966/kaggle/notebooks/Project-NeuMF/outputs_csv/neumf_dot",
    out_name="neumf_dot_1"
)